In [8]:
import numpy as np 
import pandas as pd
#import calendar
import math

# plots
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

In [9]:
filenames = ['properties/properati-AR-2016-03-01-properties-sell-six_months.csv',
            'properties/properati-AR-2016-04-01-properties-sell.csv',
            'properties/properati-AR-2016-05-01-properties-sell.csv',
            'properties/properati-AR-2016-06-01-properties-sell.csv',
            'properties/properati-AR-2016-07-01-properties-sell.csv',
            'properties/properati-AR-2016-08-01-properties-sell.csv',
            'properties/properati-AR-2016-09-01-properties-sell.csv',
            'properties/properati-AR-2016-10-01-properties-sell.csv',
            'properties/properati-AR-2016-11-01-properties-sell.csv',
            'properties/properati-AR-2016-12-01-properties-sell.csv', 
            'properties/properati-AR-2017-01-01-properties-sell.csv',
            'properties/properati-AR-2017-02-01-properties-sell.csv',
            'properties/properati-AR-2017-08-01-properties-sell-six_months.csv']

dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))
    
#Uno los dataFrame
data = pd.DataFrame()
for dataframe in dataframes:
    data = pd.concat([data, dataframe]).drop_duplicates().reset_index(drop=True)

In [10]:
data = data.loc[(data.state_name.str.contains('Buenos Aires')) | (data.state_name.str.contains('Capital Federal')),:]
data.shape
# 1.283.741 entries antes, ahora 296476, se redujeron 1/4 los datos

(115928, 27)

In [11]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...


<br><B><FONT FACE="Arial" SIZE="5">Prediccion usando XGBoosting'</FONT></B><br/>

In [12]:
#FILTRO LOS DATOS ANOMALOS

data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                          'floor', 'rooms', 'lat', 'lon']].dropna(axis= 0, how='any')
#test_float = test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2']].dropna(axis= 0, how='any')

#filtro
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 15.0) & (data_float.surface_total_in_m2 <= 1000.0)) & \
               ((data_float.surface_covered_in_m2 >= 15.0) & (data_float.surface_covered_in_m2 <= 500.0)) & \
               ((data_float.floor >= 1.0) & (data_float.floor <= 6.0)) & \
               ((data_float.rooms >= 1.0) & (data_float.rooms <= 8.0)), :]


#test_float = test_float.loc[(test_float.surface_total_in_m2 >= 42.0) & (test_float.surface_total_in_m2 <= 88.0), :]
#test_float = test_float.loc[(test_float.surface_covered_in_m2 >= 39.0) & (test_float.surface_covered_in_m2 <= 80.0), :]
#test_float = test_float.loc[(test_float.floor >= 1.0) & (test_float.floor <= 6.0), :]
#test_float = test_float.loc[(test_float.rooms >= 1.0) & (test_float.rooms <= 3.0), :]

data_float.shape

(2786, 7)

In [13]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [14]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [15]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.824/0.718


Precisión Gradient Boosting train/test  0.824/0.724


In [16]:
#Completo los valores NaN con 0 para poder predecir el csv de Kaggle
test_float = test
test_float.fillna(0, inplace=True)
test_float.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    14166 non-null object
lat                        14166 non-null float64
lon                        14166 non-null float64
surface_total_in_m2        14166 non-null float64
surface_covered_in_m2      14166 non-null float64
floor                      14166 non-null float64
rooms                      14166 non-null float64
expenses                   14166 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage

In [17]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms']])


In [18]:
#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoosting.csv', index=False)

<br><B><FONT FACE="Arial" SIZE="5">XGBoosting con lat y lon</FONT></B><br/>

In [47]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms', 'lat', 'lon']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [48]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [49]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.824/0.718
#Precisión Gradient Boosting train/test  0.618/0.787


Precisión Gradient Boosting train/test  0.889/0.881


In [50]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingLatLon.csv', index=False)

<br><B><FONT FACE="Arial" SIZE="5">Uso grid-search para estimar los parametros</FONT></B><br/>

In [22]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',  'lat', 'lon']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [29]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_test1 = {'n_estimators':list(range(1,500,1))}

gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500,\
min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),\
param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1 ,max_features='sqrt',\
#subsample=0.8, random_state=10), param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
                        
gsearch1.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=50,
             min_samples_split=500, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=10,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, ...480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499]},
       pre_dispatch='2*n_jobs', refit=True,
       scor

In [30]:
#Me fijo el resultado del parametro
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -11841600048.57080, std: 3870338427.83384, params: {'n_estimators': 1},
  mean: -10921691298.01503, std: 3844864629.90425, params: {'n_estimators': 2},
  mean: -10129110632.18837, std: 3837753778.16447, params: {'n_estimators': 3},
  mean: -9650653831.80201, std: 3859050926.21013, params: {'n_estimators': 4},
  mean: -8950614456.64144, std: 3759787091.37428, params: {'n_estimators': 5},
  mean: -8635733893.26473, std: 3738683888.65959, params: {'n_estimators': 6},
  mean: -8198551076.69595, std: 3718495076.45195, params: {'n_estimators': 7},
  mean: -7886632642.02046, std: 3719235225.97881, params: {'n_estimators': 8},
  mean: -7501511810.16714, std: 3647620745.95932, params: {'n_estimators': 9},
  mean: -7176848503.17094, std: 3550715378.03507, params: {'n_estimators': 10},
  mean: -7019029660.58363, std: 3568812761.24934, params: {'n_estimators': 11},
  mean: -6872760781.19732, std: 3590452654.45807, params: {'n_estimators': 12},
  mean: -6730515272.16886, std: 3537700460.732

In [32]:
#Encontrar los parametros max_depth y min_samples_split
#Aca ya uso n_estimators=204
param_test2 = {'max_depth':list(range(2,16,2)), 'min_samples_split':list(range(200,1001,10))}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=204, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -5401045889.09421, std: 3368044461.72373, params: {'max_depth': 2, 'min_samples_split': 200},
  mean: -5321874528.79169, std: 3308890899.65660, params: {'max_depth': 2, 'min_samples_split': 210},
  mean: -5323320126.26237, std: 3301906551.57141, params: {'max_depth': 2, 'min_samples_split': 220},
  mean: -5308342746.46211, std: 3314259311.64956, params: {'max_depth': 2, 'min_samples_split': 230},
  mean: -5451643226.61515, std: 3453968629.57617, params: {'max_depth': 2, 'min_samples_split': 240},
  mean: -5459696342.57422, std: 3466663683.11916, params: {'max_depth': 2, 'min_samples_split': 250},
  mean: -5556909274.29323, std: 3465243773.70523, params: {'max_depth': 2, 'min_samples_split': 260},
  mean: -5546414755.81134, std: 3476385388.30410, params: {'max_depth': 2, 'min_samples_split': 270},
  mean: -5521162542.79763, std: 3494085747.53594, params: {'max_depth': 2, 'min_samples_split': 280},
  mean: -5496871291.22369, std: 3457662744.77619, params: {'max_depth': 2, 'min_sa

In [37]:
#Encontrar el parametro min_samples_leaf
# 'n_estimators':204, 'max_depth': 14, 'min_samples_split': 300

param_test3 = {'min_samples_leaf':list(range(10,100,1))}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=204, min_samples_split=300, max_depth=14,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train, y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: -5297045798.83045, std: 3613733465.71381, params: {'min_samples_leaf': 10},
  mean: -5275271478.25623, std: 3415092462.28535, params: {'min_samples_leaf': 11},
  mean: -5156495666.21794, std: 3212733792.11737, params: {'min_samples_leaf': 12},
  mean: -5163682533.12453, std: 3221581690.17055, params: {'min_samples_leaf': 13},
  mean: -5032458927.23831, std: 3072960181.94785, params: {'min_samples_leaf': 14},
  mean: -4999684189.90907, std: 2962465555.27542, params: {'min_samples_leaf': 15},
  mean: -5123144194.45243, std: 3053499833.97466, params: {'min_samples_leaf': 16},
  mean: -5040935490.09819, std: 2981478802.30016, params: {'min_samples_leaf': 17},
  mean: -5035361446.20603, std: 3028452721.46038, params: {'min_samples_leaf': 18},
  mean: -4988986460.56779, std: 3030127627.80473, params: {'min_samples_leaf': 19},
  mean: -4870697120.98120, std: 3011357836.20741, params: {'min_samples_leaf': 20},
  mean: -4870261211.00545, std: 2935789199.22983, params: {'min_samples_leaf

In [40]:
#Encontrar el parametro max_features
# 'n_estimators':204, 'max_depth': 14, 'min_samples_split': 300, 'min_samples_leaf': 44

param_test4 = {'max_features':list(range(1,4,1))}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=204, max_depth=14, min_samples_split=300, min_samples_leaf=44, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train, y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: -4827706489.40967, std: 2969047976.59676, params: {'max_features': 1},
  mean: -4657529385.05661, std: 2897624847.05577, params: {'max_features': 2},
  mean: -4743130397.92881, std: 2960990437.02800, params: {'max_features': 3}],
 {'max_features': 2},
 -4657529385.056612)

In [43]:
# 'n_estimators':204, 'max_depth': 14, 'min_samples_split': 300, 'min_samples_leaf': 44, 'max_features': 2
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 204, 'max_depth': 14, 'min_samples_split': 300, 'min_samples_leaf': 44, 'max_features': 2,
          'learning_rate': 0.1}


clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

#Me fijo la precision
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))


Precisión Gradient Boosting train/test  0.829/0.851


In [46]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS.csv', index=False)